In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import requests
import json
import datetime
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import (Mail, Attachment, FileContent, FileName, FileType, Disposition, ContentId, To)
import base64

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
-- Get sendgrid_api_token from Snowflake secrets
use schema PYTHON_WORKLOADS.email_reports;
CREATE OR REPLACE function sendgrid_api_key_secret()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.9
HANDLER = 'get_secret_1'
EXTERNAL_ACCESS_INTEGRATIONS = (SENDGRID_APIS_ACCESS_INTEGRATION_EMAIL)
SECRETS = ('cred' = sendgrid_dev_API_TOKEN )
AS
$$
import _snowflake
def get_secret_1():
   secret_type = _snowflake.get_generic_secret_string('cred')
   return secret_type
$$;
grant usage on function sendgrid_api_key_secret() to role svc_engineer_prd;

In [ ]:
# Store sf api token in variable
api_secret_query = "select PYTHON_WORKLOADS.email_reports.sendgrid_api_key_secret()"
sendgrid_api_key = session.sql(api_secret_query).collect()[0][0]

In [ ]:
current_env="UAT"
errorlist = []
failed_batches = []
failed_merchseqkeys_list = []
error_msg = []
error_field = []

In [ ]:
if current_env=="PROD":
    login_url=""
    api_url=""
elif current_env=="UAT":
    login_url="https://heartlandpaymentsystems--qauat.sandbox.my.salesforce.com/services/oauth2/token"
    api_url="https://heartlandpaymentsystems--qauat.sandbox.my.salesforce.com/services/apexrest/IntegrationAPI/Note"
else:
    login_url="https://heartlandpaymentsystems--simpdev1.sandbox.my.salesforce.com/services/oauth2/token"
    api_url="https://heartlandpaymentsystems--simpdev1.sandbox.my.salesforce.com/services/apexrest/IntegrationAPI/Note"

In [ ]:
# Get Auth Token for SF
def sf_auth(salesforce_uname,salesforce_pass,salesforce_clientid,salesforce_clientsecret):
    payload = {
        'grant_type':'password',
        'username':salesforce_uname,
        'password':salesforce_pass,
        'client_id':salesforce_clientid,
        'client_secret':salesforce_clientsecret
    }
    response=requests.post(login_url,data=payload)
    response_data=response.json()
    if(response.status_code!=200):
        raise Exception(f"Failed to get access token: {response_data}")
    return response_data['access_token'],response_data['instance_url']

In [ ]:
# Post request to SalesForce
def post_notes(notes_dict,access_token,api_url,max_retries=3,backoff_factor=1):
    headers= json.loads('{"Authorization":"Bearer '+access_token+'"}')
    for post_req_attempt in range(max_retries):
        response = requests.post(api_url,headers=headers,data=notes_dict)
        print(response)
        if response.status_code == 200:
            return True,[]
        elif response.status_code == 201:
            response_list=response.json()
            print(response_list)
            for response in response_list:
                if response['error'] == "This record was not inserted because Sequence Key does not exist":
                    failed_merchseqkeys_list.append(response['merchantSequenceKey'])
                    error_msg.append(response['error'])
                    AUTH="Missing keys"
                else:
                    failed_merchseqkeys_list.append(response['merchantSequenceKey'])
                    error_msg.append(response['error'])
                    AUTH=False
            return AUTH,failed_merchseqkeys_list,error_msg
        elif response.status_code == 401:
            return "AUTH",[],[]
        else:
            print(f'Batch failed (attempt {post_req_attempt+1}/{max_retries}): {response.status_code}, {response.text}')
            time.sleep(backoff_factor * (2 ** post_req_attempt))
            return "AUTH",[],[]

    print(f"Failed after {max_retries} attempts")
    return False,[],[],[]

In [ ]:
# Connect to Salesfoce and get access token
access_token_tuple=sf_auth("","","","")
# access_token_tuple=sf_auth(salesforce_uname,salesforce_pass,salesforce_clientid,salesforce_clientsecret)
access_token=access_token_tuple[0]

In [ ]:
-- Query to fetch new notes
select merchantsequencekey,subject,memo,SHA2(CONCAT(sequencekey,merchantsequencekey,entereddate)) as hash_value 
from HCSDB_STG.dbo.memos
where entereddate between DATEADD(HOUR, -48, current_timestamp()) and current_timestamp() 
and hash_value not in (select distinct hash_value from PYTHON_WORKLOADS.SERVICING.HCSDB_NOTELOG)
and isfromsalesforce=FALSE;

In [ ]:
notes_df=cell10.to_pandas()
notes_df=notes_df.rename(columns={'MERCHANTSEQUENCEKEY':'merchantSequenceKey','SUBJECT':'subject','MEMO':'memo'})
if len(notes_df.index)>0:
    for counter in range(0,len(notes_df),100):
        notes_batch = notes_df.iloc[counter:counter+100]
        notes_dict_pd = notes_batch.to_dict(orient='records')
        notes_dict = json.dumps(notes_dict_pd)
        status,failed_list,err_msg=post_notes(notes_dict,access_token,api_url)
        if status == 'AUTH':
            try:
                access_token_tuple=sf_auth(salesforce_uname,salesforce_pass,salesforce_clientid,salesforce_clientsecret)
                access_token=access_token_tuple[0]
            except Exception as e:
                print(f'Failed to refresh access token and send batch - {str(e)}')
            finally:
                failed_batches.append(notes_batch['merchantSequenceKey'])
                auth_fail=True
    
        elif not status:
            for merchseqkey in failed_list:
                failed_batches.append(merchseqkey)
                auth_fail=False
    
        else:
            pass

In [ ]:
if len(notes_df.index)>0:
    error_concat_list=[str(field)+' - '+str(value) + ' - ' + str(key) for field,value,key in zip(error_field,error_msg,failed_list)]
    error_concat_list=list(set(error_concat_list))
    failed_batches=list(set(failed_batches))
    failed_batches_df = pd.DataFrame({'merchantSequenceKey':failed_batches})
    failed_batches_df['merchantSequenceKey']=failed_batches_df['merchantSequenceKey'].astype(str).astype(int)
    case_hash_df=notes_df[['merchantSequenceKey','HASH_VALUE']]
    df_merge=pd.merge(case_hash_df,failed_batches_df,on='merchantSequenceKey',how='left',indicator=True)
else:
    df_merge = pd.DataFrame()

In [ ]:
if not df_merge.empty:
    case_hash_df_filtered=df_merge[df_merge['_merge']=='left_only'].drop(columns='_merge')
    case_hash_df_filtered=case_hash_df_filtered[['HASH_VALUE']]
    case_hash_df_filtered['DTEADDED'] = pd.Timestamp("now")
    case_hash_df_filtered['DTEADDED'] = case_hash_df_filtered['DTEADDED'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
    case_hash_df_filtered.reset_index(drop=True, inplace=True)
    print(case_hash_df_filtered)
else:
    case_hash_df_filtered=pd.DataFrame()

In [ ]:
if len(case_hash_df_filtered.index)>0:
    session.write_pandas(case_hash_df_filtered,"HCSDB_NOTELOG",database="PYTHON_WORKLOADS",schema="SERVICING",auto_create_table=False,overwrite=False)
else:
    pass

In [ ]:
if len(failed_batches)>0:
    cc_email = ''
    
    if not auth_fail:
        sub1  = "Note objects :: Missing Merchant Sequence key"
        body1 = """Note objects were not created on the previous run - """ + pd.Timestamp("today").strftime("%m/%d/%Y") + f""". Please refer to the attached file for more details."""
        message = Mail(
        from_email='bi@heartland.us',
        to_emails=[To('arvindraj.rajakumar@e-hps.com')],
        subject=sub1,
        plain_text_content=body1
        )
        if cc_email:
            message.add_cc(cc_email)
            
    else:
        sub1  = "Note objects :: Authentication Issue"
        body1 = f"""Failed to create notes object due to an authentication issue.\nPlease refer to the attached file for the list of merchant sequence keys that were not created."""

        message = Mail(
            from_email='bi@heartland.us',
            to_emails=[To('arvindraj.rajakumar@e-hps.com')],
            subject=sub1,
            plain_text_content=body1
        )

    
    failed_id_df = pd.DataFrame({'Field - Value - Sequence Key':error_concat_list})
    base64_csv = base64.b64encode(failed_id_df.to_csv(index=False).encode()).decode()

    message.attachment = Attachment(FileContent(base64_csv),
                        FileName('Failed_Notes.csv'),
                        FileType('text/csv'),
                        Disposition('attachment'),
                        ContentId('dataframe'))


    sg = SendGridAPIClient(sendgrid_api_key)
    response = sg.send(message)